<a href="https://colab.research.google.com/github/PurvaChiniya/Sentiment_analysis/blob/master/olid_paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORT LIBRARIES

In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# DATA

In [2]:
cd /content/drive/My Drive/ml-lab

/content/drive/My Drive/ml-lab


In [3]:
data = pd.read_csv('./olid-training-v1.0.tsv',sep='\t', header=None)
msk = np.random.rand(len(data)) < 0.8
train_data = data[msk]
test_data = data[~msk]
data.head()

,0,1,2,3,4
0,id,tweet,subtask_a,subtask_b,subtask_c
1,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
2,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
3,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
4,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN


# DATA PREPROCESSING

In [4]:
import numpy as np
import pandas as pd
import re
import io
import string
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim.models import FastText

nltk.download('stopwords')
nltk.download('wordnet')

pd.options.mode.chained_assignment = None  # default='warn'

REDUNDANT_INFO = ['@USER', 'URL']

EMOJI_PATTERN = re.compile(
    u"(\ud83d[\ude00-\ude4f])|"
    u"(\ud83c[\udf00-\uffff])|"
    u"(\ud83d[\u0000-\uddff])|"
    u"(\ud83d[\ude80-\udeff])|"
    u"(\ud83c[\udde0-\uddff])"
    "+", flags=re.UNICODE)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
def lemmatize(x):
    lemmatizer = WordNetLemmatizer()
    return lemmatizer.lemmatize(x, pos='v')

In [0]:
class Preprocessing:

    @staticmethod
    def remove_unnecessary_information(x):
        """
        Remove unnecessary information from a string.
        :param x: A string.
        :return: A cleaned string.
        """
        for i in REDUNDANT_INFO:
            x = x.replace(i, '')

        # Remove emojis.
        x = EMOJI_PATTERN.sub(r'', x)

        # Word tokenizer.
        tokenizer = RegexpTokenizer(r'\w+')
        x = tokenizer.tokenize(x)

        # Remove stop words.
        stop_words = set(stopwords.words('english'))
        x = [w for w in x if w not in stop_words]
        x = [w for w in x if not w.isdigit()]
        x = [w for w in x if w not in string.punctuation]
        x = [lemmatize(w) for w in x]

        return x

    @staticmethod
    def load_embeddings(file_name):
        """
        Load the embeddings.
        :param file_name: Path to embeddings.
        :return: Embeddings.
        """
        fin = io.open(file_name, 'r', encoding='utf-8', newline='\n', errors='ignore')
        n, d = map(int, fin.readline().split())
        print("Number of words in FastText embeddings: ", n, " Dimension:", d)
        data = {}
        for line in fin:
            tokens = line.rstrip().split(' ')
            word = tokens[0]
            representation = np.asarray(tokens[1:], dtype='float32')
            data[word] = representation
        fin.close()
        return data

    @staticmethod
    def load_data(file_name):
        """
        Load data from disk.
        :param file_name: Path to the file.
        :return:
        """
        return pd.read_csv(file_name, sep="\t")

    @staticmethod
    def filter_data(data, subtask='subtask_a'):
        """
        Filter data by removing redundant information from tweets.
        :param data: Data as a pandas data frame.
        :return: Filtered data.
        """

        data.dropna(subset=[subtask], inplace=True)

        print("Labels: ", data[subtask].unique())

        if subtask == 'subtask_a':
            y = data[["subtask_a"]]
            y['subtask_a'].replace(to_replace=['OFF', 'NOT'], value=[0, 1], inplace=True)
        elif subtask == 'subtask_b':
            zero_category = data[subtask] == 'UNT'
            zero_category_data = data[zero_category]
            data = data.append([zero_category_data] * 7, ignore_index=True)
            y = data[["subtask_b"]]
            y['subtask_b'].replace(to_replace=['UNT', 'TIN'], value=[0, 1], inplace=True)
        elif subtask == 'subtask_c':
            one_category = data[subtask] == 'GRP'
            one_category_data = data[one_category]
            data = data.append([one_category_data] * 1, ignore_index=True)

            two_category = data[subtask] == 'OTH'
            two_category_data = data[two_category]
            data = data.append([two_category_data] * 5, ignore_index=True)

            y = data[["subtask_c"]]
            y['subtask_c'].replace(to_replace=['IND', 'GRP', 'OTH'], value=[0, 1, 2], inplace=True)

        X = data[["tweet"]]
        X['tweet'] = X['tweet'].map(lambda x: Preprocessing.remove_unnecessary_information(x))

        print(y[subtask].value_counts())

        return pd.Series(X['tweet']), y.values

    @staticmethod
    def filter_test_data(data):
        """
        Filter data by removing redundant information from tweets.
        :param data: Data as a pandas data frame.
        :return: Filtered data.
        """

        X = data[["tweet"]]

        X['tweet'] = X['tweet'].map(lambda x: Preprocessing.remove_unnecessary_information(x))

        return pd.Series(data['id']), pd.Series(X['tweet'])

    @staticmethod
    def prepare_data(X_train, X_test, max_tweet_length):
        """
        Prepare data by tokenizing it.
        :param X_train: Train data as an ndarray.
        :param X_test: TestA data as an ndarray.
        :param max_tweet_length: A maximum length of a tweet.
        :return: Padded train/test data, mapping of words to the number of texts they appeared, mapping of words to indices
        """

        tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token='<UNK>')
        tokenizer.fit_on_texts(X_train.ravel())
        train_words_to_indices = tokenizer.texts_to_sequences(X_train.ravel())
        test_words_to_indices = tokenizer.texts_to_sequences(X_test.ravel())

        # Add zeroes to to the tweet, if its length less than max_tweet_length.
        train_padded = tf.keras.preprocessing.sequence.pad_sequences(train_words_to_indices, maxlen=max_tweet_length,
                                                                     padding='post', truncating='post')
        test_padded = tf.keras.preprocessing.sequence.pad_sequences(test_words_to_indices, maxlen=max_tweet_length,
                                                                    padding='post', truncating='post')

        print("Shape of the train data: ", train_padded.shape)
        print("Shape of the test data: ", test_padded.shape)

        # len(tokenizer.word_docs) + 2, because of UNKNOWN and PAD.
        return train_padded, test_padded, tokenizer.word_docs, tokenizer.word_index, len(tokenizer.word_docs) + 3

    @staticmethod
    def create_embedding_matrix(word2idx, dimension, embeddings_file_name):
        """
        Create an embedding matrix.
        :param word2idx: A mapping from a word to an index.
        :param dimension: A dimension of embeddings.
        :param embeddings_file_name: Path to the file of GloVe embeddings.
        :return: An embedding matrix.
        """
        max_words = len(word2idx) + 1
        embedding_matrix = np.zeros((max_words, dimension))

        # Load GloVe embeddings.
        embeddings_data = Preprocessing.load_embeddings(embeddings_file_name)

        zeros = 1
        for word, index in word2idx.items():
            embedding_vector = embeddings_data.get(word)
            if embedding_vector is not None:
                embedding_matrix[index] = embedding_vector
            else:
                zeros += 1

        print("Shape of the embedding matrix: ", embedding_matrix.shape)
        print("{} words are not found".format(zeros))

        return embedding_matrix

    @staticmethod
    def get_tweet_embedding(data):

        model = FastText(size=300, window=5, min_count=1, seed=42, workers=2, alpha=0.025, min_alpha=0.00025)

        docs = []

        for index, tweet in data.iteritems():
            words = word_tokenize(tweet.lower())
            docs.append(words)

        model.train(docs, total_examples=model.corpus_count, epochs=10)
        return np.asarray([model.wv[tweet] for i, tweet in data.iteritems()])

    @staticmethod
    def map_indices_to_embeddings(x_data, y_data, embedding_matrix):

        mapped_data = np.zeros((x_data.shape[0] * x_data.shape[1], embedding_matrix.shape[1]))
        mapped_labels = np.zeros((mapped_data.shape[0], 1))

        index = 0
        for r in range(0, x_data.shape[0]):

            # Iterate over rows
            row = x_data[r, :]

            for c in range(0, row.size):
                # Iterate over columns
                embedding = embedding_matrix[x_data[r, c]]
                for e in range(0, embedding_matrix.shape[1]):
                    mapped_data[index, e] = embedding[e]
                mapped_labels[index, 0] = y_data[r, 0]
                index += 1

        return mapped_data, mapped_labels

In [0]:
def shuffle_batch(X, y, batch_size):
    """
    Create batches.
    :param X: Features
    :param y: Labels
    :param batch_size:
    :return:
    """
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, np.reshape(y_batch, (y_batch.shape[0],))

In [0]:
from datetime import datetime
#from Utils import shuffle_batch
#from Preprocessing import Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score as sklearn_f1_score
from tensorflow.python.ops.rnn import bidirectional_dynamic_rnn as bi_rnn

# Hyperparameters:
EMBEDDINGS_FILENAME = "./crawl-300d-2M.vec"
EMBEDDING_DIMENSION = 300
BATCH_SIZE = 128
MAX_TWEET_LENGTH = 20
EPOCHS = 100
LSTM_NEURONS = 256
NEURONS_HIDDEN_LAYER_1 = 128
RNN_LAYERS = 3
DROPOUT_KEEP_PROBABILITY = 0.5
NEURONS_SOFTMAX = 2
LOG_DIR = "./model_output"
LEARNING_RATE = 0.001
TRAINABLE_EMBEDDINGS = False
LAMBDA_L2_REG = 0.00001
SUBTASK = 'subtask_a'
import tensorflow as tf

In [9]:
pwd

'/content/drive/My Drive/ml-lab'

In [10]:
# Load and filter data.
X, y = Preprocessing.filter_data(Preprocessing.load_data("/content/drive/My Drive/ml-lab/olid-training-v1.0.tsv"), SUBTASK)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Tokenize data.
train_data, test_data, vocab_freq, word2idx, vocab_size = Preprocessing.prepare_data(x_train, x_test,
                                                                                     MAX_TWEET_LENGTH)


Labels:  ['OFF' 'NOT']
1    8840
0    4400
Name: subtask_a, dtype: int64
Shape of the train data:  (10592, 20)
Shape of the test data:  (2648, 20)


In [11]:
# Create an embedding matrix.
embedding_matrix = Preprocessing.create_embedding_matrix(word2idx, EMBEDDING_DIMENSION, EMBEDDINGS_FILENAME)

# Placeholders.
X = tf.placeholder(tf.int32, [None, MAX_TWEET_LENGTH], name="X_input")
y = tf.placeholder(tf.int64, [None], name="y_label")
keep_prob = tf.placeholder_with_default(1.0, shape=())

Number of words in FastText embeddings:  1999995  Dimension: 300
Shape of the embedding matrix:  (14862, 300)
2055 words are not found


In [13]:


# Split data.



# Define the variable that will hold the embedding.
embeddings = tf.get_variable(name="embeddings", shape=[vocab_size, EMBEDDING_DIMENSION],
                             initializer=tf.constant_initializer(embedding_matrix), trainable=TRAINABLE_EMBEDDINGS)

# Find the embeddings.
x_embedded = tf.nn.embedding_lookup(embeddings, X)
print("Input shape: ", x_embedded.shape)

# A dynamic RNN.
lstm_cells = [tf.nn.rnn_cell.LSTMCell(num_units=LSTM_NEURONS, name='lstm_cell')
              for layer in range(RNN_LAYERS)]
cells_drop = [tf.nn.rnn_cell.DropoutWrapper(cell, input_keep_prob=keep_prob)
              for cell in lstm_cells]
multi_cell = tf.nn.rnn_cell.MultiRNNCell(cells_drop)

# A bidirectional RNN is used.
outputs, states = bi_rnn(multi_cell, multi_cell, inputs=x_embedded, dtype=tf.float32)
print("RNN forward output shape: ", outputs[0].shape)
print("RNN backward output shape: ", outputs[1].shape)

outputs = tf.add(outputs[0][:, -1, :], outputs[1][:, -1, :])
print("RNN squeezed output shape: ", outputs.shape)

# A hidden layer.
hidden1 = tf.layers.dense(outputs,
                          NEURONS_HIDDEN_LAYER_1, name="hidden_1", activation='relu')
print("Hidden layer shape: ", hidden1.shape)

# A classification layer.
logits = tf.layers.dense(hidden1, NEURONS_SOFTMAX, name="softmax", activation='softmax')
print("Logits shape: ", logits.shape)

# Loss and optimizer.
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy, name="loss")

l2 = LAMBDA_L2_REG * sum([
    tf.nn.l2_loss(tf_var)
    for tf_var in tf.trainable_variables()
    if ("bias" not in tf_var.name or "carry_b" not in tf_var.name)]
)

loss += l2
print("L2 regularized loss: ", loss)

optimizer = tf.train.RMSPropOptimizer(learning_rate=LEARNING_RATE)
training_op = optimizer.minimize(loss)

# Predictions and accuracy.
predictions = tf.argmax(logits, 1, name="predictions")
correct_predictions = tf.equal(predictions, y)
accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32), name='accuracy')

# Initializer.
init = tf.global_variables_initializer()

# Saver.
saver = tf.train.Saver()

# Summary information for saving.
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
acc_summary = tf.summary.scalar('Accuracy', accuracy)
summary_op = tf.summary.merge_all()
logdir = "{}/run-{}/".format(LOG_DIR, now)

with tf.Session() as sess:
    summary_writer_train = tf.summary.FileWriter(logdir + '/train', tf.get_default_graph())
    summary_writer_test = tf.summary.FileWriter(logdir + '/test')

    init.run()
    for epoch in range(1, EPOCHS + 1):
        X_batch = None
        y_batch = None
        for X_batch, y_batch in shuffle_batch(train_data, y_train, BATCH_SIZE):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, keep_prob: DROPOUT_KEEP_PROBABILITY})

        # Accuracies after one epoch.
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: test_data, y: np.reshape(y_test, (y_test.shape[0],))})

        # Get predictions for the test set.
        _, y_pred = sess.run([accuracy, predictions], feed_dict={X: test_data, y: np.reshape(y_test, (y_test.shape[0],))})

        # Write summaries.
        summary_train_acc = acc_summary.eval(feed_dict={X: train_data, y: np.reshape(y_train, (y_train.shape[0],))})
        summary_test_acc = acc_summary.eval(feed_dict={X: test_data, y: np.reshape(y_test, (y_test.shape[0],))})
        summary_writer_train.add_summary(summary_train_acc, epoch)
        summary_writer_test.add_summary(summary_test_acc, epoch)

        print("Epoch: {} Last batch accuracy: {} Test accuracy: {} F1-Score: {}".format(epoch, acc_train, acc_test,
                                                                sklearn_f1_score(y_test, y_pred, average='macro')))

    saver.save(sess, LOG_DIR + "/tf_model")


W0813 08:03:07.559999 140117377906560 deprecation.py:323] From <ipython-input-13-50a9569e0df3>:10: LSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W0813 08:03:07.570703 140117377906560 deprecation.py:323] From <ipython-input-13-50a9569e0df3>:13: MultiRNNCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
W0813 08:03:07.572912 140117377906560 deprecation.py:323] From <ipython-input-13-50a9569e0df3>:16: bidirectional_dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell)

Input shape:  (?, 20, 300)


W0813 08:03:08.004721 140117377906560 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0813 08:03:08.014777 140117377906560 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/rnn_cell_impl.py:961: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0813 08:03:09.079304 140117377906560 deprecation.py:323] From <ipython-input-13-50a9569e0df3>:25: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions fo

RNN forward output shape:  (?, 20, 256)
RNN backward output shape:  (?, 20, 256)
RNN squeezed output shape:  (?, 256)
Hidden layer shape:  (?, 128)
Logits shape:  (?, 2)
L2 regularized loss:  Tensor("add_11:0", shape=(), dtype=float32)


W0813 08:03:10.858713 140117377906560 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/rmsprop.py:119: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Epoch: 1 Last batch accuracy: 0.6201550364494324 Test accuracy: 0.6642749309539795 F1-Score: 0.3991377354209213
Epoch: 2 Last batch accuracy: 0.6589147448539734 Test accuracy: 0.6827794313430786 F1-Score: 0.46992293063757384
Epoch: 3 Last batch accuracy: 0.7829457521438599 Test accuracy: 0.7280966639518738 F1-Score: 0.606498910243709
Epoch: 4 Last batch accuracy: 0.8062015771865845 Test accuracy: 0.7809667587280273 F1-Score: 0.737261320837348
Epoch: 5 Last batch accuracy: 0.8139534592628479 Test accuracy: 0.7711480259895325 F1-Score: 0.7081857005377701
Epoch: 6 Last batch accuracy: 0.7829457521438599 Test accuracy: 0.7749244570732117 F1-Score: 0.7438730764611754
Epoch: 7 Last batch accuracy: 0.8062015771865845 Test accuracy: 0.7768126726150513 F1-Score: 0.7192531514702376
Epoch: 8 Last batch accuracy: 0.8062015771865845 Test accuracy: 0.7802115082740784 F1-Score: 0.7169318933215781
Epoch: 9 Last batch accuracy: 0.8062015771865845 Test accuracy: 0.7794561982154846 F1-Score: 0.7330556691